In [365]:
import torch
import numpy as np
import torch.nn.functional as F
from torchvision.transforms.functional import adjust_contrast
a= torch.rand(64,64, 3) # N H W C
input = torch.stack([a,a])
parameters = torch.Tensor([0.5,0.5])

In [366]:
def relu(x):
    x_ = x.copy()
    x_[x_<0] = 0
    return x_

class AdjustContraste():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["contrast"]

    def __call__(self, list_editted, parameters):     
        editted  =  list_editted.numpy()
        mean = editted.mean()
        editted_ = (editted-mean)*(parameters[0]+1)+mean
        editted_ = relu(editted_)
        editted_ = 1-relu(1-editted_)
        return [editted_]
    
old = AdjustContraste()(a,[0.5])[0]

In [367]:
class AdjustContrast():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["contrast"]

    def __call__(self, images:torch.Tensor, parameters:torch.Tensor):
        batch_size = parameters.shape[0]
        mean = images.view(batch_size,-1).mean(1)
        mean = mean.view(batch_size, 1, 1, 1)
        parameters = parameters.view(batch_size, 1, 1, 1)
        editted = (images-mean)*(parameters+1)+mean
        editted = F.relu(editted)
        editted = 1-F.relu(1-editted)
        return editted
new = AdjustContrast()(input,torch.Tensor([0.5,0.5]))

In [368]:
(old==new[0].numpy()).all()

np.True_

In [369]:
from envs.dehaze.src import dehaze

class AdjustDehazee():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["dehaze"]

    def __call__(self, list_editted, parameters):
        editted = list_editted.numpy()
        scale = max((editted.shape[:2])) / 512.0
        omega = parameters[0]
        editted_ = dehaze.DarkPriorChannelDehaze(
            wsize=int(15*scale), radius=int(80*scale), omega=omega,
            t_min=0.25, refine=True)(editted * 255.0) / 255.0
        editted_ = relu(editted_)
        editted_ = 1-relu(1-editted_)
        return [editted_]
old = AdjustDehazee()(a,[0.5])[0]

In [399]:
class AdjustDehaze():

    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["dehaze"]

    def __call__(self, images, parameters):
        """
        Takes a batch of images where B (the last dim) is the batch size
        args:
            images: torch.Tensor # B H W C 
            parameters :torch.Tensor # N
        return:
            output: torch.Tensor #  B H W C 
        """
        assert images.dim()==4
        batch_size = parameters.shape[0]
        output = []
        for image_index in range(batch_size):
            image = images[image_index].numpy()
            scale = max((image.shape[:2])) / 512.0
            omega = float(parameters[image_index])
            editted= dehaze.DarkPriorChannelDehaze(
                wsize=int(15*scale), radius=int(80*scale), omega=omega,
                t_min=0.25, refine=True)(image * 255.0) / 255.0
            editted = torch.tensor(editted)
            editted = F.relu(editted)
            editted= 1-F.relu(1-editted)
            output.append(editted)
        output = torch.stack(output)
        return output
    
    
new = AdjustDehaze()(input,parameters)

In [371]:
(old==new.numpy()[0]).all()

np.True_

In [448]:
import cv2

class AdjustClaritye():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["clarity"]

    def __call__(self, list_editted, parameters):
        editted = list_editted.numpy()
        scale = max((editted.shape[:2])) / 512.0
        clarity = parameters[0]

        unsharped = cv2.bilateralFilter((editted*255.0).astype(np.uint8),
                                            int(32*scale), 50, 10*scale)/255.0
        editted_ = editted + (editted-unsharped) * clarity
        return [editted_]
    
old = AdjustClaritye()(a,[0.5])[0]

In [453]:

class AdjustClarity():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["clarity"]

    def __call__(self, images, parameters):
        """
        Takes a batch of images where B (the last dim) is the batch size
        args:
            images: torch.Tensor # B H W C 
            parameters :torch.Tensor # N
        return:
            output: torch.Tensor #  B H W C 
        """
        assert images.dim()==4
        batch_size = parameters.shape[0]
        output = [] 
        clarity = parameters.view(batch_size, 1, 1, 1)
        for image in images: 
            input = image.numpy()      
            scale = max((input.shape[:2])) / 512.0
            unsharped = cv2.bilateralFilter((input*255.0).astype(np.uint8),
                                                int(32*scale), 50, 10*scale)/255.0
            output.append(torch.tensor(unsharped))
        output = torch.stack(output) 
        editted_images = images + (images-output) * clarity
        
        return editted_images
    
# class AdjustClarity():
#     def __init__(self):
#         self.num_parameters = 1
#         self.window_names = ["parameter"]
#         self.slider_names = ["clarity"]

#     def __call__(self, images, parameters):
#         assert images.dim()==4
#         batch_size = parameters.shape[0]
#         output = [] 
        
#         for image_index,image in enumerate(images): 
#             clarity = float(parameters[image_index])
#             input = image.numpy()      
#             scale = max((input.shape[:2])) / 512.0
#             unsharped = cv2.bilateralFilter((input*255.0).astype(np.uint8),
#                                                 int(32*scale), 50, 10*scale)/255.0
#             editted = input + (input-unsharped) * clarity
#             output.append(torch.tensor(editted))

#         output = torch.stack(output) 

#         return output
new = AdjustClarity()(input,parameters)

In [455]:
(old==new.numpy()[1]).all()

np.True_

# exposure

In [636]:
def sigmoid_inverse(y):
    epsilon = 10**(-3)
    y = F.relu(y-epsilon)+epsilon
    y = 1-epsilon-F.relu((1-epsilon)-y)
    y = (1/y)-1
    output = -np.log(y.numpy())
    return torch.tensor(output)

class SigmoidInverse():

    def __init__(self):
        self.num_parameters = 0

    def __call__(self, images):
        return sigmoid_inverse(images)
new_sig_inv = SigmoidInverse()

def old_sigmoid_inverse(y):
    epsilon = 10**(-3)
    y_ = y.copy()
    y_ = relu(y_-epsilon)+epsilon
    y_ = 1-epsilon-relu((1-epsilon)-y_)
    y_ = (1/y_)-1
    output = -np.log(y_)
    return output

In [637]:
(old_sigmoid_inverse(a.numpy())==new_sig_inv (input)[0].numpy()).all()

np.True_

In [638]:
class AdjustExposuree():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["exposure"]

    def __call__(self, list_sigmoid_inversed, parameters):
        exposure = parameters[0]
        return [old_sigmoid_inverse(list_sigmoid_inversed)+ exposure*5]
old = AdjustExposuree()(a.numpy(),[0.5])

In [ ]:
class AdjustExposure():
    def __init__(self):
        self.num_parameters = 1
        self.window_names = ["parameter"]
        self.slider_names = ["exposure"]

    def __call__(self, list_sigmoid_inversed, parameters):
        exposure = parameters[0]
        return [list_sigmoid_inversed[0] + exposure*5]